## Генерация полифонической музыки с кондишнингом

Импортируем torch и numpy:

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
import random

Сделаем также пользовательский импорт

In [2]:
from decode_patterns import data_conversion

In [3]:
data_height = 64
drum_width = 14
melody_width = 36
data_width = drum_width + melody_width
data_size = data_height*data_width
patterns_file = "decode_patterns/patterns.pairs.tsv"

Загружаем датасет

In [4]:
# import dataset
drum, bass = data_conversion.make_lstm_dataset_conditioning(height=data_height,
                                                            limit=1000,
                                                            patterns_file=patterns_file,
                                                            mono=False)
# print(drum[0])
# drum, bass = np.array(drum), np.array(bass)
# print(drum[0])

# define shuffling of dataset
def shuffle(A, B, p=0.8):
    # take 80% to training, other to testing
    AB = list(zip(A, B))
    L = len(AB)
    pivot = int(p*L)
    random.shuffle(AB)
    yield [p[0] for p in AB[:pivot]]
    yield [p[1] for p in AB[:pivot]]
    yield [p[0] for p in AB[pivot:]]
    yield [p[1] for p in AB[pivot:]]
    
    
# we can select here a validation set
drum, bass, drum_validation, bass_validation = shuffle(drum, bass)
    
# and we can shuffle train and test set like this:
# drum_train, bass_train, drum_test, bass_test = shuffle(drum, bass)

In [5]:
bass[16]

NumpyImage(image=array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), tempo=480, instrument=26, denominator=4, min_note=72)

Модель определим в самом простом варианте, который только можно себе представить -- как в примере с конечным автоматом

In [6]:
# Encoder = LSTM
# Decoder = FCNN
class DrumNBass_LSTM_to_FCNN(nn.Module):
    def __init__(self, bass_height, bass_width):
        super(DrumNBass_LSTM_to_FCNN, self).__init__()
        # save data parameters
        self.bass_height = bass_height
        self.bass_width = bass_width
        self.bass_size = bass_height*bass_width
        self.condition_size = 2 # размер подмешиваемого conditioning
        self.embedding_size = 1 # размер латентного пространства (на каждый отсчёт)
                                # ЛУЧШЕ НЕ МЕНЯТЬ ЭТОТ ПАРАМЕТР С 1, придётся переписывать код!
        # one input neuron, one output neuron, one layer in LSTM block
        self.input_size = 14
        self.lstm_hidden_size = 6
        self.lstm_layer_count = 1
        self.lstm = nn.LSTM(self.input_size, self.lstm_hidden_size, self.lstm_layer_count)
        self.lstm_embed_layer = nn.Linear(self.lstm_hidden_size, self.embedding_size)
        
        self.decoder_layer1 = nn.Linear(self.bass_height + self.condition_size, 4)
        self.decoder_layer2 = nn.Linear(4, 48)
        self.decoder_layer3 = nn.Linear(48 + self.condition_size, 512)
        self.decoder_layer4 = nn.Linear(512, self.bass_size)
        self.sigm = nn.Sigmoid()
        
    def encoder(self, input):
        # пусть в input у нас приходит вектор размерности (32, 128, 14)
        # где имеется 32 примера (минибатч) по 128 отсчётов, 14 значений в каждом (барабанная партия)
        # Тогда его надо транспонировать в размерность (128, 32, 14)
        input = input.transpose(0,1)
        output, _ = self.lstm(input)
        output = self.sigm(self.lstm_embed_layer(output))
        return output
    
    def decoder(self, input, cond):
        output = torch.cat((input, cond), axis=1) # добавляем conditioning
        output = self.sigm(self.decoder_layer1(output))
        output = self.sigm(self.decoder_layer2(output))
        output = torch.cat((output, cond), axis=1) # добавляем ещё conditioning
        output = self.sigm(self.decoder_layer3(output))
        output = self.sigm(self.decoder_layer4(output))
        return output
    
    def forward(self, input):
        images = torch.tensor(list(map(lambda p: p.image, input)), dtype=torch.float)
        result = self.encoder(images)
        # избавляемся от лишней размерности (embedding_size=1), чтобы получить вектор из lstm
        # размером с высоту изображения
        result = result.squeeze().transpose(0,1)
        # добавляем conditioning
        conditionings = torch.tensor(list(map(lambda p: [p.tempo, p.instrument], input)), dtype=torch.float)
        conditionings = conditionings
        result = self.decoder(result, conditionings)
        return result.view((-1, self.bass_height, self.bass_width))

In [7]:
# часть обучения
dnb_lstm = DrumNBass_LSTM_to_FCNN(data_height, melody_width)

criterion = nn.MSELoss()

# оценим также и разнообразие мелодии по её.. дисперсии?)
# def melody_variety(melody):
#     return 1/(1 + (melody.sum(axis=2) > 1).int())
    
# criterion = nn.NLLLoss() # -- этот товарищ требует, чтобы LSTM выдавал классы,
# criterion = nn.CrossEntropyLoss() # и этот тоже
# (числа от 0 до C-1), но как всё-таки его заставить это делать?...
# optimizer = optim.SGD(dnb_lstm.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(dnb_lstm.parameters(), lr=0.001)

Посмотрим, как модель форвардится на один пример

In [8]:
dnb_lstm.forward([drum_validation[16], drum_validation[14], drum_validation[43]]).size()

torch.Size([3, 64, 36])

In [9]:
bass_validation[16].image.shape

(64, 36)

Найденные баги и их решения:

https://stackoverflow.com/questions/56741087/how-to-fix-runtimeerror-expected-object-of-scalar-type-float-but-got-scalar-typ

https://stackoverflow.com/questions/49206550/pytorch-error-multi-target-not-supported-in-crossentropyloss/49209628

https://stackoverflow.com/questions/56243672/expected-target-size-50-88-got-torch-size50-288-88

In [10]:
epoch_count = 500
batch_size = 32
shuffle_every_epoch = True
    
if shuffle_every_epoch:
    print(f"shuffle_every_epoch is on")
else:
    print(f"shuffle_every_epoch is off")
    # shuffle train and test set:
    drum_train, bass_train, drum_test, bass_test = shuffle(drum, bass)
        
for epoch in range(epoch_count):  # loop over the dataset multiple times
    print(f"Epoch #{epoch}")
    if shuffle_every_epoch:
        # shuffle train and test set:
        drum_train, bass_train, drum_test, bass_test = shuffle(drum, bass)
        
    examples_count = len(drum_train)
    examples_id = 0
    
    running_loss = 0.0
    runnint_count = 0
    batch_id = 0
    while examples_id < examples_count:
        batch_drum_train = drum_train[examples_id:examples_id + batch_size]
        batch_bass_train = bass_train[examples_id:examples_id + batch_size]
        
        batch_bass_train_raw = torch.tensor(list(map(lambda p: p.image, batch_bass_train)), dtype=torch.float)
#         batch_bass_train_raw = batch_bass_train_raw.transpose(0, 1)
        # transpose нужен БЫЛ для обмена размерности батча и размерности шагов

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        bass_outputs = dnb_lstm(batch_drum_train).squeeze()
#         bass_outputs = bass_outputs.reshape(bass_outputs.size()[0], -1)
#         batch_bass_train = batch_bass_train.reshape(batch_bass_train.size()[0], -1)
#         print(f"bass_outputs:{bass_outputs.size()} batch_bass_train: {batch_bass_train}")
#         print(f"bass_outputs:{bass_outputs} batch_bass_train: {batch_bass_train}")
        
        # loss = criterion(bass_outputs, batch_bass_train.long())
        loss = criterion(bass_outputs, batch_bass_train_raw)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        runnint_count += 1
        period = 5
        if batch_id % period == 0 or examples_id + batch_size >= examples_count:
            print('[%d, %5d] train loss: %.7f' %
                  (epoch + 1, batch_id + 1, running_loss / runnint_count))
            running_loss = 0.0
            runnint_count = 1
            
        # update batch info
        examples_id += batch_size
        batch_id += 1
        
    # here we can insert measure error on test set

#should check accuracy on validation set
print('Finished Training')

shuffle_every_epoch is on
Epoch #0
[1,     1] train loss: 0.2578243
[1,     6] train loss: 0.1030614
[1,    11] train loss: 0.0238111
[1,    16] train loss: 0.0106624
[1,    20] train loss: 0.0087067
Epoch #1
[2,     1] train loss: 0.0121254
[2,     6] train loss: 0.0085282
[2,    11] train loss: 0.0083864
[2,    16] train loss: 0.0081099
[2,    20] train loss: 0.0076956
Epoch #2
[3,     1] train loss: 0.0118281
[3,     6] train loss: 0.0082910
[3,    11] train loss: 0.0079239
[3,    16] train loss: 0.0079213
[3,    20] train loss: 0.0080420
Epoch #3
[4,     1] train loss: 0.0109282
[4,     6] train loss: 0.0081549
[4,    11] train loss: 0.0081571
[4,    16] train loss: 0.0083848
[4,    20] train loss: 0.0077897
Epoch #4
[5,     1] train loss: 0.0099802
[5,     6] train loss: 0.0080573
[5,    11] train loss: 0.0078968
[5,    16] train loss: 0.0081859
[5,    20] train loss: 0.0079075
Epoch #5
[6,     1] train loss: 0.0095020
[6,     6] train loss: 0.0080933
[6,    11] train loss: 0.0080

KeyboardInterrupt: 

In [11]:
with torch.no_grad():
    bass_outputs = dnb_lstm(drum_train)

In [12]:
result = bass_outputs.squeeze().int()
result

tensor([[[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        ...,

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0,

Попробуем сохранить результаты работы сети. На anaconda нет mido, поэтому сохраняем результаты работы просто в массивчик npy... Однако, как альтернатива, его можно поставить чере pip в conda:
https://github.com/mido/mido/issues/198

In [15]:
import mido
from decode_patterns.data_conversion import build_track, DrumMelodyPair, NumpyImage, Converter


converter = Converter((data_height, data_width))

batch_drum = drum_train + drum_test + drum_validation
batch_drum = drum_validation
batch_bass = bass_train + bass_test + bass_validation
batch_bass = drum_validation
with torch.no_grad():
    bass_outputs = dnb_lstm(batch_drum)
    bass_outputs = ((bass_outputs.squeeze() + 1) / 2 > 0.55).int()
    
    for i in range(len(batch_drum)):
            
        img_dnb = np.concatenate((batch_drum[i].image,bass_outputs[i]), axis=1)
        numpy_pair = NumpyImage(np.array(img_dnb)
                                , batch_drum[i].tempo
                                , batch_drum[i].instrument
                                , 1
                                , batch_drum[i].min_note)
        pair = converter.convert_numpy_image_to_pair(numpy_pair)
        mid = build_track(pair, tempo=pair.tempo)
        mid.save(f"midi/npy/sample{i+1}.mid")

Наблюдения:
1. Нейронная сеть выдаёт по одной барабанной партии одну басовую партию. Следовательно, страдает разнообразие генерируемой музыки
2. (следствие из 1) В датасете (скорее всего) имеются типичные ритмы, которым соответствуют абсолютно разные мелодии на разных инструментах. Это сильно сбивает с толку нейросеть. Как следствие, на уникальных ритмах нейросеть переобучается, а на типичных -- не понимает, какую мелодию сгенерировать

Предложения:
1. Придумать способ разделить пары (каким-то образом) в данной ситуации, возможно добавить жанр или ещё что..
2. Разметить музыку тональностями -- это может упростить обучение нейросети
3. Добавить случайность в латентное пространство